In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
# Load the molecule (for example purposes, we'll assume a simple peptide)
# Replace with actual molecule loading code
molecule = Chem.MolFromSmiles('CC(=O)NCCC(=O)NCC')
molecule = Chem.AddHs(molecule)
AllChem.EmbedMolecule(molecule)
AllChem.MMFFOptimizeMolecule(molecule)

0

In [ ]:
atom = molecule.GetAtomWithIdx(0)
sum([bond.GetBondTypeAsDouble() for bond in atom.GetBonds()])

In [10]:
# Function to get atom positions
def get_atom_positions(molecule):
    conf = molecule.GetConformer()
    atom_positions = {}
    for atom in molecule.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        atom_positions[atom.GetIdx()] = np.array([pos.x, pos.y, pos.z])
    return atom_positions

# Function to find specific atoms by their atomic number and bonding partners
def find_atoms(molecule, atomic_num, bonded_atomic_num):
    atoms = []
    for atom in molecule.GetAtoms():
        if atom.GetAtomicNum() == atomic_num:
            for neighbor in atom.GetNeighbors():
                if neighbor.GetAtomicNum() == bonded_atomic_num:
                    atoms.append(atom)
    return atoms

# Function to find specific atoms by their atomic number and bonding partners
def find_donors_and_acceptors(molecule):

    donors = []
    acceptors = []

    for atom in molecule.GetAtoms():

        if atom.GetAtomicNum() == 7:    
            if 1 in [atom_2.GetAtomicNum() for atom_2 in atom.GetNeighbors()]:
                    donors.append(atom)

            if sum([bond.GetBondTypeAsDouble() for bond in atom.GetBonds()]) < 4:
                acceptors.append(atom)

        elif atom.GetAtomicNum() == 8:
            if 1 in [atom_2.GetAtomicNum() for atom_2 in atom.GetNeighbors()]:
                donors.append(atom)
            
            if sum([bond.GetBondTypeAsDouble() for bond in atom.GetBonds()]) < 3:
                acceptors.append(atom)

    return donors, acceptors


# Calculate distances between all relevant pairs
def calculate_distance(atom1, atom2):
    return np.linalg.norm(atom_positions[atom1.GetIdx()] - atom_positions[atom2.GetIdx()])

# Get all atom positions
atom_positions = get_atom_positions(molecule)

#get the donors and acceptors
donors, acceptors = find_donors_and_acceptors(molecule)

# Calculate the electrostatic energy matrix
electrostatic_energy_matrix = np.zeros((len(donors), len(acceptors)))

for i, donor in enumerate(donors):    
    for j, acceptor in enumerate(acceptors):
        donor_acceptor_r = calculate_distance(donor, acceptor)
        CONVERSION_FACTOR_KJ = 1388.93  # kJ mol^-1 Å^-1
        energy = CONVERSION_FACTOR_KJ *  donor.GetProp('_TriposAtomCharges') * acceptor.GetProp('_TriposAtomCharges') / donor_acceptor_r

        if energy > 5: 
            electrostatic_energy_matrix[i, j] = energy 

/var/folders/5j/_bnmcm0j4115r6hln80zmksc0000gn/T/ipykernel_43073/2563154982.py:56: RuntimeWarning: divide by zero encountered in scalar divide
  E = 0.42 * 0.20 * 33.2 * (1/r_ON + 1/r_CH - 1/r_OH - 1/r_CN)
